In [1]:
import sys
import os
import yaml
import json

sys.path.append('../../')

from constants import ROOT_DIR

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load and preprocess data
def load_data(file_path):
    df = pd.read_csv(file_path, header=None, names=['event_type', 'agent_id', 'context'])
    return df

def preprocess_data(df):
    le_event = LabelEncoder()
    le_agent = LabelEncoder()
    le_context = LabelEncoder()
    
    df['event_type_encoded'] = le_event.fit_transform(df['event_type'])
    df['agent_id_encoded'] = le_agent.fit_transform(df['agent_id'])
    df['context_encoded'] = le_context.fit_transform(df['context'])
    
    return df, le_event, le_agent, le_context

# Create sequences
def create_sequences(df, sequence_length):
    sequences = []
    targets = []
    
    for i in range(len(df) - sequence_length):
        seq = df.iloc[i:i+sequence_length]
        target = df.iloc[i+sequence_length]
        
        sequences.append(seq[['event_type_encoded', 'agent_id_encoded', 'context_encoded']].values)
        targets.append([target['event_type_encoded'], target['agent_id_encoded'], target['context_encoded']])
    
    return np.array(sequences), np.array(targets)

# Build model
def build_model(input_shape, num_classes):
    input_event = Input(shape=input_shape)
    input_agent = Input(shape=input_shape)
    input_context = Input(shape=input_shape)
    
    concat = Concatenate(axis=-1)([input_event, input_agent, input_context])
    hidden = Dense(64, activation='relu')(concat)
    flattened = Flatten()(hidden)
    
    output_event = Dense(num_classes[0], activation='softmax', name='event_type')(flattened)
    output_agent = Dense(num_classes[1], activation='softmax', name='agent_id')(flattened)
    output_context = Dense(num_classes[2], activation='softmax', name='context')(flattened)
    
    model = Model(inputs=[input_event, input_agent, input_context],
                  outputs=[output_event, output_agent, output_context])
    
    model.compile(optimizer='adam',
                  loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'],
                  loss_weights=[1.0, 1.0, 1.0],
                  metrics=['accuracy', 'accuracy', 'accuracy'])
    
    return model

# Main pipeline
def main():
    # Load and preprocess data
    df = load_data('..\..\data\processed/games/tic-tac-toe/1k_single_agent.csv')
    df, le_event, le_agent, le_context = preprocess_data(df)
    
    # Create sequences
    sequence_length = 3
    X, y = create_sequences(df, sequence_length)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Prepare inputs and outputs
    X_train_event = X_train[:, :, 0]
    X_train_agent = X_train[:, :, 1]
    X_train_context = X_train[:, :, 2]
    
    X_test_event = X_test[:, :, 0].reshape(X_test.shape[0], X_test.shape[1], 1)
    X_test_agent = X_test[:, :, 1].reshape(X_test.shape[0], X_test.shape[1], 1)
    X_test_context = X_test[:, :, 2].reshape(X_test.shape[0], X_test.shape[1], 1)
    
    y_train_event = to_categorical(y_train[:, 0], num_classes=len(le_event.classes_))
    y_train_agent = to_categorical(y_train[:, 1], num_classes=len(le_agent.classes_))
    y_train_context = to_categorical(y_train[:, 2], num_classes=len(le_context.classes_))
    
    y_test_event = to_categorical(y_test[:, 0], num_classes=len(le_event.classes_))
    y_test_agent = to_categorical(y_test[:, 1], num_classes=len(le_agent.classes_))
    y_test_context = to_categorical(y_test[:, 2], num_classes=len(le_context.classes_))
    
    # Build and train model
    model = build_model((sequence_length,), 
                        [len(le_event.classes_), len(le_agent.classes_), len(le_context.classes_)])

    model.fit([X_train_event, X_train_agent, X_train_context],
              [y_train_event, y_train_agent, y_train_context],
              validation_data=([X_test_event, X_test_agent, X_test_context],
                                [y_test_event, y_test_agent, y_test_context]),
              epochs=1, batch_size=32)

    # Evaluate model
    results = model.evaluate(
        [X_test_event, X_test_agent, X_test_context],
        [y_test_event, y_test_agent, y_test_context]
    )
    
    # Print evaluation results
    print("Test Results:")
    for metric_name, value in zip(model.metrics_names, results):
        print(f"{metric_name}: {value:.4f}")

    # Predict next move for a custom log
    custom_log = np.array([
        [le_event.transform(['GAME_START'])[0], le_agent.transform(['system'])[0], le_context.transform(['New Game'])[0]],
        [le_event.transform(['MOVE'])[0], le_agent.transform(['X'])[0], le_context.transform(['2,2'])[0]],
        [le_event.transform(['MOVE'])[0], le_agent.transform(['O'])[0], le_context.transform(['2,0'])[0]]
    ])

    custom_event = custom_log[:, 0].reshape(1, 3)
    custom_agent = custom_log[:, 1].reshape(1, 3)
    custom_context = custom_log[:, 2].reshape(1, 3)

    predictions = model.predict([custom_event, custom_agent, custom_context])

    predicted_event = le_event.inverse_transform([np.argmax(predictions[0])])
    predicted_agent = le_agent.inverse_transform([np.argmax(predictions[1])])
    predicted_context = le_context.inverse_transform([np.argmax(predictions[2])])

    print(f"Predicted next move: {predicted_event[0]}, {predicted_agent[0]}, {predicted_context[0]}")

if __name__ == "__main__":
    main()

242/242 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - agent_id_accuracy: 0.5827 - context_accuracy: 0.1403 - event_type_accuracy: 0.6942 - loss: 4.6271 - val_agent_id_accuracy: 0.8779 - val_context_accuracy: 0.2131 - val_event_type_accuracy: 0.8914 - val_loss: 3.0682
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 798us/step - agent_id_accuracy: 0.8733 - context_accuracy: 0.2144 - event_type_accuracy: 0.8854 - loss: 3.0942
Test Results:
loss: 3.0682
compile_metrics: 0.8779
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Predicted next move: MOVE, X, 1,0
